In [21]:
%reload_ext autoreload
%autoreload 2

import h5py
N_Cs = 2

with h5py.File('../dxtb/dxtb-gpu/gpu-cpu_analysis/rdkit/alkanes_data_500.hdf5', 'r') as f:
    for mol_name, data in f.items():
        if mol_name == f"alkane_{N_Cs}_carbons":
            atomic_numbers = data['atomic_numbers'][:]
            coordinates = data['coordinates'][:]

print(f"Number of carbon atoms in {mol_name}: {N_Cs}")
print(f"Nb of atoms: {len(atomic_numbers)}")

Number of carbon atoms in alkane_9_carbons: 2
Nb of atoms: 8


In [22]:
import dxtb
from dxtb._src.typing import DD
import torch
from dxtb.config import ConfigCache

batch_size = 64

print(f"Number of carbon atoms in {mol_name}: {N_Cs}")
print(f"Nb of atoms: {len(atomic_numbers)}")
print(f"batch_size: {batch_size}")

dd = {"device": torch.device("cuda:0"), "dtype": torch.float64}
numbers = torch.tensor(atomic_numbers, device=dd["device"], dtype=torch.int32)
positions = torch.tensor(coordinates, **dd).requires_grad_()
charges = torch.tensor(0.0, **dd)
# numbers = torch.stack([numbers] * batch_size)
# positions = torch.stack([positions] * batch_size).requires_grad_()
# charges = torch.zeros((batch_size,), device=dd["device"], dtype=dd["dtype"])

results = {}

Number of carbon atoms in alkane_9_carbons: 2
Nb of atoms: 8
batch_size: 64


In [23]:
opts = {"scf_mode": "full", "batch_mode": 0, "int_driver": "libcint", "maxiter":10000}

calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts, timer=True)
calc.opts.cache = ConfigCache(enabled=False, density=True, fock=True, overlap=False)
dxtb.timer.reset()
e = calc.get_energy(positions, chrg=charges)
dxtb.timer.start("Forces autograd")
forces = torch.autograd.grad(e, positions, retain_graph=True)[0]
dxtb.timer.stop("Forces autograd")
dxtb.timer.print(v=0)

results[f"e_{opts['scf_mode']}"] = e
results[f"forces_{opts['scf_mode']}"] = forces
results[f"Fgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.cache["fock"].sum(), positions, retain_graph=True)[0]
results[f"Pgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.get_density(positions, chrg=charges).sum(), positions, retain_graph=True)[0]

# For reconnect modes
scf_charges = calc.get_charges(positions, chrg=charges)
scf_charge_mode = opts["scf_mode"]

Total Energy: -2.02464984361025 Hartree.


Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.006          10.74
 - DispersionD3             0.005          83.78
 - Repulsion                0.001          12.11
 - Halogen                  0.000           2.58
Integrals                   0.003           5.48
 - Overlap                  0.002          47.75
 - Core Hamiltonian         0.002          51.37
SCF                         0.031          51.23
 - Interaction Cache        0.001           2.59
 - Potential                0.006          17.81
 - Fock build               0.000           1.24
 - Diagonalize              0.009          29.06
 - Density                  0.001           4.64
 - Charges                  0.002           5.37
Forces autograd             0.018          29.62
------------------------------------------------
Sum                         0.059          97.08
Total    

In [ ]:
opts = {"scf_mode": "implicit", "batch_mode":0, "int_driver": "libcint", "maxiter":10000}

calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts, timer=True)
calc.opts.cache = ConfigCache(enabled=False, density=True, fock=True, overlap=False)
dxtb.timer.reset()
e = calc.get_energy(positions, chrg=charges)
dxtb.timer.start("Forces autograd")
forces = torch.autograd.grad(e, positions, retain_graph=True)[0]
dxtb.timer.stop("Forces autograd")
dxtb.timer.print(v=0)

results[f"e_{opts['scf_mode']}"] = e
results[f"forces_{opts['scf_mode']}"] = forces
results[f"Fgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.cache["fock"].sum(), positions, retain_graph=True)[0]
results[f"Pgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.get_density(positions, chrg=charges).sum(), positions, retain_graph=True)[0]

# For reconnect modes
scf_charges = calc.get_charges(positions, chrg=charges)
scf_charge_mode = opts["scf_mode"]


# Gradchecker

In [25]:
from torch.autograd import gradcheck
from dxtb import OutputHandler
from dxtb.config import ConfigCache

OutputHandler.verbosity = 0

# Inputs (must be float64)
positions_d = positions.detach().double().requires_grad_()
charges_d = charges.double()

calc.opts.cache = ConfigCache(enabled=False, fock=True, density=True)

def run_gradcheck(fn, inputs):
    # gradcheck assumes float64 and requires_grad=True
    inputs = tuple(i.detach().double().requires_grad_() for i in inputs)
    passed = gradcheck(fn, inputs, eps=1e-6, atol=1e-3, rtol=1e-3, nondet_tol=1e-5)
    print(f"Gradcheck passed: {passed}")

# Energy (scalar output)
def energy_fn(pos):
    return calc.get_energy(pos, chrg=charges_d)

print("Energy")
run_gradcheck(energy_fn, (positions_d,))

# Fock (matrix output)
def fock_fn(pos):
    _ = calc.get_energy(pos, chrg=charges_d)  # populate cache
    return calc.cache["fock"]

print("Fock")
run_gradcheck(fock_fn, (positions_d,))

# Density (matrix output)
def density_fn(pos):
    return calc.get_density(pos, chrg=charges_d)

print("Density")
run_gradcheck(density_fn, (positions_d,))


Energy
Gradcheck passed: True
Fock


GradcheckError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[ 9.1775e-04,  0.0000e+00,  0.0000e+00,  ..., -7.7745e-04,
         -7.9411e-12, -9.3888e-03],
        [-2.4456e-02,  0.0000e+00,  0.0000e+00,  ...,  1.8232e-03,
          8.7683e-12,  1.0153e-02],
        [-1.4742e-02,  0.0000e+00,  0.0000e+00,  ...,  9.0220e-04,
          1.4517e-11,  1.7044e-02],
        ...,
        [ 5.4271e-03,  0.0000e+00,  0.0000e+00,  ...,  2.7158e-03,
          7.6143e-12,  1.0155e-02],
        [ 5.5978e-03,  0.0000e+00,  0.0000e+00,  ...,  4.3776e-03,
          7.2094e-12,  9.3194e-03],
        [-3.4624e-03,  0.0000e+00,  0.0000e+00,  ..., -6.0885e-03,
         -5.2169e-12, -7.1223e-03]], device='cuda:0', dtype=torch.float64)
analytical:tensor([[-3.1492e-05,  0.0000e+00,  0.0000e+00,  ..., -1.1486e-05,
         -1.3174e-13, -2.0574e-04],
        [ 8.3921e-04,  0.0000e+00,  0.0000e+00,  ..., -1.2915e-04,
         -1.0590e-12, -1.4031e-03],
        [ 5.0586e-04,  0.0000e+

In [ ]:
from torch.autograd import gradcheck
from dxtb import OutputHandler

OutputHandler.verbosity = 0

def run_gradcheck(fn, inputs):
    inputs = tuple(i.detach().requires_grad_() for i in inputs)
    passed = gradcheck(fn, inputs)
    print(f"Gradcheck passed: {passed}")

# Functions must return tuple of tensor outputs in float64
def energy_fn(pos):
    return (calc.get_energy(pos, chrg=charges),)

def fock_fn(pos):
    calc.get_energy(pos, chrg=charges)  # populate cache
    return (calc.cache["fock"].sum(),)

def density_fn(pos):
    return (calc.get_density(pos, chrg=charges).sum(),)

# Inputs
positions_d = positions.detach().requires_grad_()
charges_d = charges

print("Energy")
run_gradcheck(energy_fn, (positions_d,))

print("Fock")
run_gradcheck(fock_fn, (positions_d,))

print("Density")
run_gradcheck(density_fn, (positions_d,))


In [ ]:
from torch.autograd import gradcheck
from torch.autograd.gradcheck import _get_numerical_jacobian, _as_tuple
from dxtb import OutputHandler

OutputHandler.verbosity = 0

def compare_grads(fn, inputs):
    # Prepare input
    inputs = tuple(i.detach().requires_grad_() for i in _as_tuple(inputs))
    output = fn(*inputs)
    output = _as_tuple(output)

    # Compute autograd
    autograd_grads = torch.autograd.grad(output, inputs, grad_outputs=[torch.ones_like(o) for o in output], retain_graph=True)

    # Compute numerical
    numerical_grads = _get_numerical_jacobian(fn, inputs, eps=1e-6)

    # Print comparison
    for i, (a, n) in enumerate(zip(autograd_grads, numerical_grads)):
        n_tensor = n[0][0]  # FIXED: Unwrap twice
        print(f"[Input {i}] max(abs diff): {(a - n_tensor).abs().max().item():.2e}")
        # print(f"Autograd:\n{a}\nNumerical:\n{n_tensor}")

def energy_fn(pos):
    return calc.get_energy(pos, chrg=charges)
print("Energy")
compare_grads(energy_fn, (positions,))

def fock_fn(pos):
    calc.get_energy(pos, chrg=charges)
    return calc.cache["fock"].sum()
print("Fock")
compare_grads(fock_fn, (positions,))

def density_fn(pos):
    return calc.get_density(pos, chrg=charges).sum()
print("Density")
compare_grads(density_fn, (positions,))